# 12-2. 사례별 시계열 데이터 계산하기

#### 에볼라 최초 발병일 계산하기.

In [ ]:
import pandas as pd

ebola = pd.read_csv('./data/country_timeseries.csv')
print(ebola.info())

In [ ]:
print(ebola.iloc[-5:, :5]) #가장 마지막 데이터 5개 ~ 최종 행, 최초 5개 열 확인: 데이터 기간이 역순으로 정렬되어있음을 확인.

In [ ]:
#min 메서드로 최초 발병일 찾기.
ebola['date_dt'] = pd.to_datetime(ebola['Date'])
print(ebola['date_dt'].min())
print(type(ebola['date_dt'].min()))

In [ ]:
#ebola 진행 정도 파악: Date 열에서 최초 발병일 차감.
firsttime = ebola['date_dt'].min()
ebola['outbreak_d'] = ebola['date_dt'] - firsttime
print(ebola['outbreak_d'])

#### 파산한 은행 개수 계산.

 - 분기별 파산한 은행 개수 확인 + 그래프 시각화

In [ ]:
banks = pd.read_csv("./data/banklist.csv")
print(banks.head())

In [ ]:
#Closing date, Updated Date는 문자열이므로 datetime object로 변환
banks_no_dates = pd.read_csv('./data/banklist.csv')
print(banks_no_dates.info())

banks = pd.read_csv('./data/banklist.csv', parse_dates = [5, 6])  #행 이름을 직접 입력할 수도 있고, 인덱스 번호로 줄 수도 있음.
print(banks.info())

In [ ]:
#dt 접근자와 quarter 속성을 이용하면 은행이 파산한 분기를 알 수 있음.
#year, quarter 속성을 이용하여 은행이 파산한 연도와 분기를 새로운 열로 추가.

banks['closing_quarter'], banks['closing_year'] = banks['Closing Date'].dt.quarter, banks['Closing Date'].dt.year
print(banks.head())

In [ ]:
#groupby 메서드: 연도별 파산한 은행 개수 확인
closing_year = banks.groupby(['closing_year']).size()
print(closing_year)
print(type(closing_year))

In [ ]:
#연도별, 분기별 파산한 은행 개수 확인(연도별로 그룹화 한 다음 분기별로 다시 그룹화하여 출력)
closing_year_q = banks.groupby(['closing_year', 'closing_quarter']).size()
print(closing_year_q)
print(type(closing_year_q))

In [ ]:
#연도별, 분기별 파산 은행 개수 그래프 시각화.
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax = closing_year.plot()
plt.show()

fig, ax = plt.subplots()
ax = closing_year_q.plot()
plt.show()

# fig = plt.figure()
# ax1 = fig.add_subplot(1, 2, 1)   #이렇게 할 경우 ax1은 연속형 데이터라 출력이 되지만,
# ax2 = fig.add_subplot(1, 2, 2)   #ax2는 groupby할 때 2개의 기준이 적용되었으므로 출력이 안 됨. 연속적인 데이터를 넣으라는 오류를 출력함.

# ax1.plot(closing_year)
# ax2.plot(closing_year_q)
# plt.show()


In [ ]:
#### 테슬라 주식 데이터로 시간 계산하기.
#아나콘다에서 pip install pandas-datareader 설치.
#1)데이터 다운로드: get_data_quanal 메서드에 TSLA라는 문자열 전달.
#2)data 폴더 안에 저장하기.
import pandas as pd
import pandas_datareader as pdr

pd.core.common.is_list_like = pd.api.types.is_list_like

#tesla 변수에 데이터프레임 저장.
tesla = pdr.get_data_quandl('TSLA', api_key = 'mrs1DZTZskGXwd2QC3pD')  #https://www.quandl.com에 가입하여 접속 api_key를 생성하고, 받은 key를 입력하라.

#tesla의 데이터프레임을 파일로 저장.
tesla.to_csv("./data/tesla_stock_quandl.csv")

In [ ]:
#데이터 확인하기.
#tesla 데이터프레임의 Date 열은 문자열로 저장되어있음 --> datetime 오브젝트로 자료형을 변환해야 시간 계산을 할 수 있음.
print(tesla.head())

tesla = pd.read_csv("./data/tesla_stock_quandl.csv", parse_dates = [0]) #0번째인 Date열을 인자에 전달.
print(tesla.info())

In [ ]:
#dt 접근자와 boolean 추출로 2010년 6월 데이터만 추출
print(tesla.loc[(tesla.Date.dt.year == 2010) & (tesla.Date.dt.month == 6)])

 - datetime 오브젝트와 인덱스: DatetimeIndex 
  * datetime 오브젝트를 데이터프레임의 인덱스로 설정 -> 원하는 시간의 데이터를 바로 추출할 수 있음.

In [ ]:
#Date열을 tesla의 인덱스로 지정하는 방법.
tesla.index = tesla['Date']
print(tesla.index)

In [ ]:
print(tesla['2015'].iloc[:5, :5])  #iloc: 데이터의 행 번호를 중심으로 데이터 출력 
                                   #loc: 데이터프레임의 인덱스 번호를 중심으로.

In [ ]:
print(tesla['2010-06'].iloc[:, :5])

 - 시간 간격과 인덱스: TimedeltaIndex
  * 일정 기간만큼의 데이터를 출력하기 위해 시간 간격을 인덱스로 지정하는 것.

In [ ]:
#Date열 - Date열의 최솟값(2010-06-29): 데이터를 수집한 이후에 흐른 시간을 알 수 있음.
tesla['ref_date'] = tesla['Date'] - tesla['Date'].min() #새 열로 추가
print(tesla.head())

In [ ]:
#시간 간격을 이용한 데이터 추출.
tesla.index = tesla['ref_date']
print(tesla.iloc[:5, :5])

In [ ]:
print(tesla['5 days':].iloc[:5, :5])  #데이터 수집 이후 최초 5일의 데이터 추출. #4 days는 데이터가 없으므로 출력되지 않음.

 - 시간 범위와 인덱스
  * 임의로 시간 범위를 생성하여 인덱스를 지정하면, 특정 일의 누락된 데이터도 포함시켜 데이터를 살펴볼 수 있음.

In [ ]:
ebola = pd.read_csv("./data/country_timeseries.csv", parse_dates = [0])
print(ebola.iloc[:5, :5]) #최초 5개 데이터 중 2015-01-05 일부 데이터가 누락됨.

In [ ]:
print(ebola.iloc[-5:, :5]) #마지막 5개 데이터도 누락됨.

In [ ]:
#date_range 메서드: 141231 ~ 150105 사이의 DatetimeIndex 생성!
head_range = pd.date_range(start = '2014-12-31', end = '2015-01-05')
print(head_range)

In [ ]:
#원본 데이터 손상을 방지하기 위해 ebola의 최초 5개 데이터를 추출하여 새 데이터프레임 생성.
ebola_5 = ebola.head()
ebola_5.index = ebola_5['Date'] #Date 열을 인덱스로 먼저 지정하지 않으면 오류가 발생함!
ebola_5.reindex(head_range)     #반드시 Date열을 먼저 인덱스로 지정한 다음, 새로 생성한 DatetimeIndex인 head_range에 해당하는 시간 범위를 인덱스로 지정해야 함.
print(ebola_5.iloc[:5, :5])

 - 시간 범위의 주기 설정하기.
 - 시간 범위의 인덱스인 DatetimeIndex 자료형에는 freq 속성이 포함되어 있음.
 - freq 속성을 지정하면 시간 간격을 조절하여 DatetimeIndex를 만들 수 있음.
 - freq 속성값으로 사용할 수 있는 시간 주기.

        B: 평일만 포함.
        C: 사용자가 정의한 평일만 포함
        D: 달력 일자 단위
        W: 주간 단위
        M: 월 마지막 날만 포함
        SM: 15일과 월 마지막 날만 포함
        BM: M 주기의 값이 휴일이면 제외하고 평일만 포함
        CBM: BM에 사용자 정의 평일을 적용
        MS: 월 시작일만 포함
        SMS: 월 시작일과 15일만 포함
        BMS: MS 주기의 값이 휴일이면 제외하고 평일만 포함
        CBMS: BMS에 사용자 정의 평일을 적용
        Q: 3, 6, 9, 12월 분기 마지막 날만 포함
        BQ: 3, 6, 9, 12월 분기 마지막 날이 휴일이면 제외하고 평일만 포함
        QS: 3, 6, 9, 12월 분기 시작일만 포함
        BQS: 3, 6, 9, 12월 분기 시작일이 휴일이면 제외하고 평일만 포함
        A: 년의 마지막 날만 포함
        BA: 년의 마지막 날이 휴일이면 제외하고 평일만 포함
        AS: 년의 시작일만 포함
        BAS: 년의 시작일이 휴일이면 제외하고 평일만 포함
        BH: 평일을 시간 단위로 포함(09:00 ~ 16:00)
        H: 시간 단위로 포함(00:00 ~ 00:00)
        T: 분 단위 포함
        S: 초 단위 포함
        L: 밀리초 단위 포함
        U: 마이크로초 단위 포함
        N: 나노초 단위 포함

In [ ]:
#date_range 메서드의 freq 인자값을 B로 설정하여 평일만 포함시킨 DatetimeIndex
print(pd.date_range('2017-01-01', '2017-01-07', freq = 'B'))

 - 시간 범위 수정하고 데이터 밀어내기: shift 메서드

In [ ]:
#시간의 흐름에 따른 속도, 추이 등을 파악하기 위해 시작 시점을 통일하는 것.
import matplotlib.pyplot as plt

ebola.index = ebola['Date']

fig, ax = plt.subplots()
ax = ebola.iloc[0:, 1:].plot(ax = ax)
ax.legend(fontsize = 7, loc = 2, borderaxespad = 0.)
plt.show() #각 나라마다 에볼라 발병일이 다르므로 그래프가 그려지는 위치도 제각각.

In [ ]:
#각 나라의 발병일을 가장 처음 에볼라가 발병한 Guinea와 동일한 위치로 옮긴 상태에서 에볼라 확산 속도 측정
#ex.Liberia는 발병일을 4만큼 옮겨야 한다.
ebola_sub = ebola[['Day', 'Cases_Guinea', 'Cases_Liberia']]
print(ebola_sub.tail(10))

In [ ]:
#그래프 그리기
ebola = pd.read_csv("./data/country_timeseries.csv", parse_dates = ['Date'])
print(ebola.head().iloc[:, :5])
print(ebola.tail().iloc[:, :5])

In [ ]:
#Date열을 인덱스로 지정한 다음, Date열의 최댓값과 최솟값으로 시간 범위를 생성하여 new_idx에 저장.
#이렇게 하면 날짜가 아예 없었던 데이터의 인덱스를 생성할 수 있음.
ebola.index = ebola['Date']
new_idx = pd.date_range(ebola.index.min(), ebola.index.max())
print(new_idx) #시간 순서가 ebola 데이터셋의 역순으로 생성되어있음. 따라서 정렬시켜준다.

new_idx = reversed(new_idx)

In [ ]:
#reindex로 새로 생성한 new_idx를 인덱스로 지정.
ebola = ebola.reindex(new_idx)  #2015-01-01 같이 ebola에 아예 없던 날짜가 추가됨.
print(ebola.head().iloc[:, :5])
print(ebola.tail().iloc[:, :5])

In [ ]:
#각 나라의 에볼라 발병일 옮기기.
#last_valid_index, first_valid_index: 유효한 값이 있는 첫 번째와 마지막 인덱스를 반환.
last_valid = ebola.apply(pd.Series.last_valid_index)
print(last_valid)

first_valid = ebola.apply(pd.Series.first_valid_index)
print(first_valid)

In [ ]:
#(에볼라가 처음 발병한 날 - 각 나라의 에볼라 발병일)만큼 옮기면 각 나라의 에볼라 발병일을 동일선상으로 맞출 수 있음.
earliest_date = ebola.index.min()
print(earliest_date)

shift_values = last_valid - earliest_date
print(shift_values)

In [ ]:
#각 나라의 에볼라 발병일 옮기기: shift 메서드로 모든 열의 값을 shift_values만큼 옮김.
ebola_dict = {}
for idx, col in enumerate(ebola):
    d = shift_values[idx].days
    shifted = ebola[col].shift(d)
    ebola_dict[col] = shifted

In [ ]:
#딕셔너리인 ebola_dict를 데이터프레임으로 변환.
ebola_shift = pd.DataFrame(ebola_dict)

In [ ]:
print(ebola_shift.tail())

In [ ]:
ebola_shift.index = ebola_shift['Day']
ebola_shift = ebola_shift.drop(['Date', 'Day'], axis = 1)
print(ebola_shift.tail())

In [ ]:
#지금까지 만든 데이터프레임을 그래프로 그리기.
fig, ax = plt.subplots()
ax = ebola_shift.iloc[:, :].plot(ax = ax)
ax.legend(fontsize = 7, loc = 2, borderaxespad = 0.)
plt.show()